In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import numpy as np
import lightgbm as lgb
# from calc_uauc import uAUC
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from scipy import sparse
import os
import warnings
from imblearn.under_sampling import RandomUnderSampler
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 100)
tqdm.pandas(desc='pandas bar')


def get_count(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'count')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df


def get_sum(df, column, feature):
    df['idx'] = range(len(df))
    temp = df.groupby(column)['userid'].agg([(feature, 'sum')]).reset_index()
    df = df.merge(temp)
    df = df.sort_values('idx').drop('idx', axis=1).reset_index(drop=True)
    return df


def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [2]:
#train = reduce_mem(train)
test = pd.read_csv('./phm_test.csv', header=None)
test.columns = ['time','gun_no','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
test = reduce_mem(test)

503.54 Mb, 165.94 Mb (67.05 %)


In [3]:
train = pd.DataFrame(columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up'])
#train.columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
#'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
for i in os.listdir("./new_train"):
    temp = pd.read_pickle('./new_train/'+i)
    train = pd.concat([train, temp[temp["W_Error"] == "E016"]])
    
#print(train.head())
#train = pd.read_pickle('./new_train/04-30.pkl')
#train.columns = ['time','gun_no','W_Error','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force','W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
#'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']
train_positive = pd.read_pickle('./new_train/05-18.pkl')
train = pd.concat([train, train_positive])
train['W_Error'].value_counts()


0       688128
0       272639
E003     68572
E011     34803
E016      2595
E029         1
Name: W_Error, dtype: int64

In [4]:
train['W_Error'][train['W_Error']==0]='0'
# train['task1_label'] = 1
train['task1_label'] = train['W_Error'].apply(lambda x:1 if x=='E016' else 0)
train['task1_label'].value_counts()

0    1064143
1       2595
Name: task1_label, dtype: int64

In [5]:
drop_columns = ['time', 'gun_no', 'area']
train.drop(drop_columns, axis=1, inplace=True)
test.drop(drop_columns, axis=1, inplace=True)
train = train.drop_duplicates()

train['task1_label'].value_counts()

0    609632
1      1501
Name: task1_label, dtype: int64

In [ ]:
from sklearn.model_selection import StratifiedKFold
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
oof_lgb = np.zeros(len(train))
feature_importance_df = pd.DataFrame()
col = [tmp_col for tmp_col in train.columns if tmp_col not in ['W_Error', 'task1_label']]
#print(train[col].head())
X_train = train[col].values
# print(train[col].info())
y_train = train['task1_label'].values
# prediction = np.zeros(len(train))
rus = SMOTEENN(random_state=0)
X_train, y_train = rus.fit_resample(X_train, y_train)
print(len(X_train))


In [14]:
param = {'num_leaves': 48,
         # 'min_data_in_leaf': 30,
         'objective': 'binary',
         'learning_rate': 0.02,
         # 'max_depth': 9,
         # "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         # "bagging_freq": 1,
         "bagging_fraction": 0.9,
         # "bagging_seed": 11,
         "metric": "auc",
         # "lambda_l1": 0.1,
         # "lambda_l2": 0.1,
         "verbosity": -1}

def f2_score(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    print(precision, recall)
    f2_score = 5 * precision * recall / (4 * precision + recall)
    return f2_score

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 2000
    clf = lgb.train(param, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=20,
                    early_stopping_rounds=100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = col
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
# oof_lgb2 += clf.predict(df_remove[col].values, num_iteration=clf.best_iteration)
# oof_lgb_final = np.argmax(oof_lgb, axis=1)
#     predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
print('feature importance:')
print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))

fold n°1
Training until validation scores don't improve for 100 rounds
[20]	training's auc: 0.999964	valid_1's auc: 0.999934
[40]	training's auc: 0.99996	valid_1's auc: 0.999934
[60]	training's auc: 0.999972	valid_1's auc: 0.999934
[80]	training's auc: 0.999977	valid_1's auc: 0.999934
[100]	training's auc: 0.999985	valid_1's auc: 0.999934
[120]	training's auc: 0.999992	valid_1's auc: 0.999956
[140]	training's auc: 0.999992	valid_1's auc: 0.999967
[160]	training's auc: 0.999993	valid_1's auc: 1
[180]	training's auc: 0.999999	valid_1's auc: 1
[200]	training's auc: 0.999999	valid_1's auc: 1
[220]	training's auc: 1	valid_1's auc: 1
[240]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[152]	training's auc: 0.999992	valid_1's auc: 1
fold n°2
Training until validation scores don't improve for 100 rounds
[20]	training's auc: 0.999968	valid_1's auc: 0.999513
[40]	training's auc: 0.999974	valid_1's auc: 0.999712
[60]	training's auc: 0.999981	valid_1's auc: 0.999701
[80]	tr

In [15]:
predict = clf.predict(test, num_iteration=clf.best_iteration)
predict[predict > 0.3] = 1
predict[predict < 0.3] = 0
print(sum(predict))
result = pd.DataFrame(predict, dtype='int8')
result.info()
result.to_csv("./predict_e016.csv", header=0, index=0)

534.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   0       int8 
dtypes: int8(1)
memory usage: 2.9 MB


In [14]:
params = {'booster': 'gbtree',
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'gamma': 1,
            'min_child_weight': 1.5,
            'max_depth': 5,
            'lambda': 10,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'colsample_bylevel': 0.7,
            'eta': 0.04,
            'tree_method': 'exact',
            'seed': 2020,
            'nthread': 36,
            "silent": True,
                      }



for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = xgb.DMatrix(X_train[trn_idx] , label=y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx] , label=y_train[val_idx])

    num_round = 2000
    watchlist = [(trn_data, 'train'),(val_data, 'eval')]
    clf = xgb.train(params, trn_data, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = col
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
# oof_lgb2 += clf.predict(df_remove[col].values, num_iteration=clf.best_iteration)
# oof_lgb_final = np.argmax(oof_lgb, axis=1)
#     predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
print('feature importance:')
print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))

fold n°1
[09:48:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.95586	eval-auc:0.95602
[200]	train-auc:0.99991	eval-auc:0.99978
[400]	train-auc:1.00000	eval-auc:0.99994
[544]	train-auc:1.00002	eval-auc:0.99992


TypeError: predict() got an unexpected keyword argument 'num_iteration'